In [198]:
"""
@authors: faurand, chardes, ehagensieker
"""
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np 

#load the mnist dataset
(train_ds, test_ds), ds_info = tfds.load('mnist', split=['train', 'test'], as_supervised=True, with_info = True)

In [199]:
import math
import datetime

# in a notebook, load the tensorboard extension, not needed for scripts
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [200]:
def prepare_mnist_data(task,batch_size,mnist):
  
  #flatten the images into one dimensional vector
  mnist = mnist.map(lambda img, target: (tf.reshape(img, (-1,)), target))
  #convert data from uint8 to float32
  mnist = mnist.map(lambda img, target: (tf.cast(img, tf.float32), target))
  #normalization to have input of range [-1,1]
  mnist = mnist.map(lambda img, target: ((img/128.)-1., target))
  #create one-hot vector targets
  #mnist = mnist.map(lambda img, target: (img, tf.one_hot(target, depth=10)))
  mnist = tf.data.Dataset.zip((mnist.shuffle(2000), mnist.shuffle(2000)))
  #mnist = mnist.map(lambda zip(pic1,pic2): (pic1[0],pic2[0],pic1[1],pic2[1]))
  mnist = calc(task,mnist)
  #mnist = mnist.map(lambda img, target: (img, target ))
  #cache this progress in memory - improve performance
  mnist = mnist.cache()
  #shuffle, batch, prefetch
  mnist = mnist.shuffle(1000)
  mnist = mnist.batch(batch_size)
  mnist = mnist.prefetch(20)

  #return preprocessed dataset
  return mnist

#apply the preprocessing to both data sets
batch_size = 32
train_1 = prepare_mnist_data(1,batch_size,train_ds)
test_1 = prepare_mnist_data(1,batch_size,test_ds)

train_2 = prepare_mnist_data(2,batch_size,train_ds)
test_2 = prepare_mnist_data(2,batch_size,test_ds)

In [201]:
def calc(task,mnist):
  if task == 1:
    mnist = mnist.map(lambda pic1,pic2: (pic1[0],pic2[0],tf.cast((pic1[1] + pic2[1] >= 5), tf.int32)))
    return mnist
  if task == 2:
    mnist = mnist.map(lambda pic1,pic2: (pic1[0],pic2[0], tf.cast((pic1[1]-pic2[1]), tf.int32)))
    return mnist




In [202]:
for i in train_2.take(1):
  print("pic1:", i[0], "\npic2: ", i[1], "\ntarget: ",i[2], i[2].shape)

pic1: tf.Tensor(
[[-1. -1. -1. ... -1. -1. -1.]
 [-1. -1. -1. ... -1. -1. -1.]
 [-1. -1. -1. ... -1. -1. -1.]
 ...
 [-1. -1. -1. ... -1. -1. -1.]
 [-1. -1. -1. ... -1. -1. -1.]
 [-1. -1. -1. ... -1. -1. -1.]], shape=(32, 784), dtype=float32) 
pic2:  tf.Tensor(
[[-1. -1. -1. ... -1. -1. -1.]
 [-1. -1. -1. ... -1. -1. -1.]
 [-1. -1. -1. ... -1. -1. -1.]
 ...
 [-1. -1. -1. ... -1. -1. -1.]
 [-1. -1. -1. ... -1. -1. -1.]
 [-1. -1. -1. ... -1. -1. -1.]], shape=(32, 784), dtype=float32) 
target:  tf.Tensor(
[ 7 -6  4  5  5  2  8 -6  3  2  6 -3  3 -3 -6  1  4 -3 -1 -3  5  1 -1 -3
  3 -4  5  7  1  1  0 -6], shape=(32,), dtype=int32) (32,)


In [203]:
"""
@authors: faurand, chardes, ehagensieker
"""

class MyModel(tf.keras.Model):
  """
  The model we are using for predicting the outcome 
  """ 
  def __init__(self, task=1):
    """
    Initializing the model 
    """
    #inherit from parent class
    super(MyModel, self).__init__()
    
    if task==1:
      self.metrics_list = [tf.keras.metrics.Mean(name = 'loss'),tf.keras.metrics.BinaryAccuracy(name = "acc")]
      self.loss_function = tf.keras.losses.BinaryCrossentropy()
      self.dense1 = tf.keras.layers.Dense(256, activation=tf.nn.relu)
      self.dense2 = tf.keras.layers.Dense(256, activation=tf.nn.relu)
      #self.dense3 = tf.keras.layers.Dense(128, activation=tf.nn.relu)
      #self.dense1 = tf.keras.layers.Dense(10, activation=tf.nn.relu)
      self.out_layer = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)

    elif task==2:
      self.metrics_list = [tf.keras.metrics.Mean(name = 'loss'),tf.keras.metrics.MeanSquaredLogarithmicError(name = "MSLE")]
      self.loss_function = tf.keras.losses.MeanSquaredError()
      self.dense1 = tf.keras.layers.Dense(256, activation= tf.nn.relu)
      self.dense2 = tf.keras.layers.Dense(256, activation= tf.nn.relu)
      # self.dense3 = tf.keras.layers.Dense(256, activation= tf.nn.relu)
      # self.dense4 = tf.keras.layers.Dense(256, activation= tf.nn.relu)
      self.out_layer = tf.keras.layers.Dense(1, activation= "linear")

    else: print("Exception")

    #self.learning_rate = 0.001
    self.optimizer = tf.keras.optimizers.Adam()

  @tf.function 
  def call(self, images):
    """
    how to forward the images through the layer
    """
    img1, img2 = images 

    img1_x = self.dense1(img1)
    img1_x = self.dense2(img1_x)
    # img1_x = self.dense3(img1_x)
    # img1_x = self.dense4(img1_x)

    img2_x = self.dense1(img2)
    img2_x = self.dense2(img2_x)
    # img2_x = self.dense3(img2_x)
    # img2_x = self.dense4(img2_x)

    combined = tf.concat([img1_x, img2_x], axis = 1)
    out = self.out_layer(combined)

    return out

  @property
  def metrics(self):
    return self.metrics_list


  def reset_metric(self):
    for metric in self.metrics: 
      metric.reset_states()

  @tf.function 
  def train_step(self, data):
    img1, img2, t = data

    with tf.GradientTape() as tape:
      output = self((img1, img2), training = True)
      loss = self.loss_function(t, output)
    
    gradients = tape.gradient(loss, self.trainable_variables)
    self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

    self.metrics[0].update_state(loss)
    
    for metric in self.metrics[1:]:
          metric.update_state(t,output)

    return {metric.name: metric.result() for metric in self.metrics}

  @tf.function
  def test_step(self, data):
    img1, img2, t = data

    output = self((img1,img2), training = False)
    loss = self.loss_function(t, output)

    self.metrics[0].update_state(loss)

    for metric in self.metrics[1:]:
          metric.update_state(t,output)
    
    return {metric.name: metric.result() for metric in self.metrics}


    


In [204]:
import tqdm
import pprint 

def training_loop(model, train_ds, val_ds, epochs, train_summary_writer, val_summary_writer): 
  #iterate over given amount of epochs
  for epoch in range(epochs): 
    print(f"Epoch {epoch}: ")

    #train on all batches of the training data
    for data in tqdm.tqdm(train_ds, position = 0, leave = True):

      metrics = model.train_step(data)

      with train_summary_writer.as_default(): 
        for metric in model.metrics: 
          tf.summary.scalar(f"{metric.name}", metric.result(), step=epoch)

    # print the metrics
    print([f"Train {key}: {value.numpy()}" for (key, value) in metrics.items()])

    #reset metric 
    model.reset_metrics()
  
    #evaluation on validation set
    for data in val_ds:
      metrics = model.test_step(data)

      with val_summary_writer.as_default():
        for metric in model.metrics:
          tf.summary.scalar(f"{metric.name}", metric.result(), step=epoch)
      
    # print the metrics
    print([f"Test {key}: {value.numpy()}" for (key, value) in metrics.items()])

    #reset metric
    model.reset_metric()

    print("\n")
      
    

In [205]:
def run(task, train_summary_writer, val_summary_writer):
  ### Hyperparameters
  num_epochs = 5
  batch_size = 32
  if task == 1:
    train = prepare_mnist_data(1,batch_size,train_ds)
    test = prepare_mnist_data(1,batch_size,test_ds)
  if task == 2:
    train = prepare_mnist_data(2,batch_size,train_ds)
    test = prepare_mnist_data(2,batch_size,test_ds)
    
  train_dataset = train
  test_dataset = test

  # Initialize the model.
  model = MyModel(task)

  training_loop(model,train_dataset,test_dataset,num_epochs,train_summary_writer, val_summary_writer)

In [206]:
config_name = "config_name"
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

train_log_path = f"logs/{config_name}/{current_time}/train"
val_log_path = f"logs/{config_name}/{current_time}/val"

#log writer 
train_summary_writer = tf.summary.create_file_writer(train_log_path)

val_summary_writer = tf.summary.create_file_writer(val_log_path)


run(2, train_summary_writer, val_summary_writer)

Epoch 0: 


100%|██████████| 1875/1875 [00:41<00:00, 45.68it/s] 


['Train loss: 2.9105515480041504', 'Train MSLE: 0.1512480080127716']
['Test loss: 1.7750355005264282', 'Test MSLE: 0.0992751345038414']


Epoch 1: 


100%|██████████| 1875/1875 [00:16<00:00, 116.49it/s]


['Train loss: 1.3685848712921143', 'Train MSLE: 0.07520443201065063']
['Test loss: 1.3474905490875244', 'Test MSLE: 0.0782305896282196']


Epoch 2: 


...

['Train loss: 1.080844521522522', 'Train MSLE: 0.05996792018413544']
['Test loss: 1.1865911483764648', 'Test MSLE: 0.065712109208107']


Epoch 3: 


100%|██████████| 1875/1875 [00:16<00:00, 112.78it/s]


['Train loss: 0.9202974438667297', 'Train MSLE: 0.05052749067544937']
['Test loss: 1.045397162437439', 'Test MSLE: 0.05946068465709686']


Epoch 4: 


100%|██████████| 1875/1875 [00:20<00:00, 91.62it/s] 


['Train loss: 0.7884718775749207', 'Train MSLE: 0.04384637996554375']
['Test loss: 1.0270785093307495', 'Test MSLE: 0.05637187883257866']


